In [2]:
# import all the various libraries
import pandas as pd
import json
from zipfile import ZipFile
from bs4 import BeautifulSoup
from azure.cosmos import CosmosClient
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import os

In [12]:
# instantiate the cosmos db client
url = os.environ['COSMOS_URI']
key = os.environ['COSMOS_KEY']

client = CosmosClient(url, credential=key)

# name file variable
file = 'ianna.xlsx'
xls = pd.ExcelFile(file)

# scrape the excel sheet for sheet names
with ZipFile(file) as zipped_file:
    summary = zipped_file.open(r'xl/workbook.xml').read()
soup = BeautifulSoup(summary, "xml")
# store the sheet names in a list
sheets = [sheet.get("name") for sheet in soup.find_all("sheet")]

# declare some variables
data_map = []
dict = {}
inc = 0

# create a data map that would be used elsewhere
for i in range(len(sheets)):
    inc += 1
    dict['id'] = inc
    dict['filter'] = sheets[i]
    data_map.append(dict.copy())
    
# create a json reponse that would be used to store the data in cosmos
data = {}
for i in range(len(sheets)):
    df = pd.read_excel(xls, sheets[i])
    result = json.loads(json.dumps(list(df.T.to_dict().values())))
    data[sheets[i]] = result
    
# create id's for all the objects
final_data = {}
for x in data:
    for i in range(len(data_map)):
        if data_map[i]['filter'] == x:
            inc = 0
            temp_lst = data[x]
            for j in range(len(temp_lst)):
                inc += 1
                temp_lst[j]['id'] = inc
            final_data[data_map[i]['filter']] = temp_lst

In [14]:
final_data['IA_Filter_ER_NC']

[{'Entity_Ruler': 'High Court',
  'Label': 'ORG',
  'NC': 'credit providers',
  'Impacted_Area': 'PBB',
  'id': 1}]

In [ ]:
os.environ['SBSA_URI'] = 'https://cdb-san-complianceregtech.documents.azure.com:443/'
os.environ['SBSA_KEY'] = '5ZMI4sYTObLLRTEEshJ6sItmUXR0EcKjwqtnXlValOzeVrjBRP8Pn4UrRLZMHWbM8GcgEuB18O5h0mgcGuiG6Q=='

In [4]:
# instantiate the cosmos db client
url = os.environ['SBSA_URI']
key = os.environ['SBSA_KEY']

client = CosmosClient(url, credential=key)

db = client.get_database_client('AlertsDB')

container = db.get_container_client('AlertsContainer')

item_list = list(container.read_all_items(max_item_count=10))
item_list

# item_list = list(container.read_all_items(max_item_count=10))

#     print('Found {0} items'.format(item_list.__len__()))

#     for doc in item_list:
#         print('Item Id: {0}'.format(doc.get('id')))


KeyError: 'SBSA_URI'